In [1]:
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10

In [1]:
import socket
import numpy as np
import time
from PIL import Image, ImageFilter
import io

In [38]:
#импульсный шум
def noise(image, prob=0.05):
    rnd = np.random.rand(image.shape[0], image.shape[1])
    noisy = image.copy()
    noisy[rnd < prob] = 0 
    noisy[rnd > 1 - prob] = 255
    return noisy


In [39]:
#Фильтр
def median_filter(a): 
    b = a.copy() 
    for i in range(2, len(a)-1):
        for j in range(2, len(a[i])-1):
            t=[0, 0, 0, 0, 0, 0, 0, 0, 0]
            t[0], t[1], t[2], t[3], t[4], t[5], t[6], t[7], t[8] = a[i-1][j-1], a[i-1][j], a[i-1][j+1], a[i][j-1], a[i][j], a[i][j+1], a[i+1][j-1], a[i+1][j], a[i+1][j+1]
            for k in range(8): 
                for l in range(8-k):
                    if t[l]>t[l+1]:
                        t[l], t[l+1] = t[l+1], t[l]
                    
            b[i][j]=t[(int)(len(t)/2)] 
    return b

In [4]:
#из байт в RGB
def bytesToRGB(img_bytes):
    image = Image.open(io.BytesIO(img_bytes))
    return np.array(image)

In [98]:
HOST = '127.0.0.1'
PORT = np.random.randint(100, 65431)
print(PORT)

41834


In [99]:
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('', PORT))
server.listen()

while True:
    print("Ожидаем соединения")
    conn, addr = server.accept()
    try:
        if conn:
            mess = conn.recv(4_194_304)
            mess_de = mess
            print(f'Сообщение получено: {mess_de[:5]}')
            print('----------------------------------------------------------')
            file = open('server_images/to_server.jpg', mode='wb')
            file.write(mess_de)
            file.close()
            print('Изображение записано')

            #код для создания шума
            Im1 = open('server_images/to_server.jpg', mode='rb')
            im1 = Im1.read(4_194_304)
            im1 = bytesToRGB(im1)
            im1 = noise(im1)
            print('Шум добавлен')
            
            im = Image.fromarray(im1)
            buf = io.BytesIO()
            im.save(buf, format='PNG')
            byte_im = buf.getvalue()
            img_file = open('server_images/to_server1.png', 'wb')
            img_file.write(byte_im)
            img_file.close()
            Im1.close()
            print('Изображение с шумом записано')
            print('----------------------------------------------------------')
            
            #код для фильтрации изображения
            client_file = open('server_images/to_server1.png', 'rb')
            client_im = client_file.read(4_194_304)
            client_file.close()
            cl_ar = bytesToRGB(client_im)
            cl_ar1 = cl_ar.reshape(cl_ar.shape[0]*cl_ar.shape[1]*cl_ar.shape[2])
            cl_ar1 = median_filter(cl_ar1.reshape(cl_ar1.shape[0]//3, 3))
            cl_ar1 = cl_ar1.reshape(1280, 1920, 3)
            cl_im = Image.fromarray(cl_ar1)
            cl_buf = io.BytesIO()
            cl_im.save(cl_buf, format='PNG')
            cl_byte = cl_buf.getvalue()
            cl_fin = open('server_images/to_server_final.png', mode='wb')
            cl_fin.write(cl_byte)
            cl_fin.close()
            print('Фильтр использован для изображения')
            
            to_client = open('server_images/to_server_final.png', mode='rb')
            mess_new = to_client.read(4_194_304)
            to_client.close()
            conn.sendall(mess_new)
            print('----------------------------------------------------------')
            print("Изображение преобразовано и отправлено")
            
    finally:
        conn.close()
        print("Соединение разорвано")
        break

Ожидаем соединения
Сообщение получено: b'\x89PNG\r'
----------------------------------------------------------
Изображение записано
Шум добавлен
Изображение с шумом записано
----------------------------------------------------------
Фильтр использован для изображения
----------------------------------------------------------
Изображение преобразовано и отправлено
Соединение разорвано
